# Overview

In this notebook, we'll load in a year of Planet Mosaic Quads into an `xarray`, check out the data using `holoviews`, and run some simple distributed computations powered by `Dask` and `Kubernetes`

# 00. Configure Dask-Kubernetes Cluster

In [1]:
from dask.distributed import Client, progress
from dask_kubernetes import KubeCluster
cluster = KubeCluster(n_workers=40)
cluster

/anaconda3/envs/pyviz/lib/python3.6/site-packages/dask/config.py:168: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  data = yaml.load(f.read()) or {}
/anaconda3/envs/pyviz/lib/python3.6/site-packages/distributed/config.py:20: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  defaults = yaml.load(f)


ModuleNotFoundError: No module named 'dask_kubernetes'

# 01. Load Mosaic-Quad Dataframes


Read in mosaics and quands datasets generated from script. Pertains to AOI-TOI at runtime.

In [3]:
import pandas as pd
pd.set_option('display.max_columns', None)

# Load
mosaics_df = pd.read_csv('../data/mosaics/mosaic_info.csv')
quad_df = pd.read_csv('../data/mosaics/quad_info.csv')

# Join datasets on mosaic ID and add column
mosaic_quad_df = pd.merge(quad_df, mosaics_df, left_on='mosaic_id', right_on='id', suffixes=('_quad', '_mosaic'))
mosaic_quad_df['time'] = pd.to_datetime(mosaic_quad_df['first_acquired'])

# Limit to one quad
single_quad_df = mosaic_quad_df[mosaic_quad_df['id_quad']=='327-1256']
single_quad_df.head()

,Unnamed: 0_quad,bbox_quad,download_link,id_quad,items_link,mosaic_id,percent_covered,self_link_quad,thumbnail,Unnamed: 0_mosaic,_links,bbox_mosaic,coordinate_system,datatype,first_acquired,grid,id_mosaic,interval,item_types,last_acquired,level,name,product_type,quad_download,self_link_mosaic,quads_link,tiles_link,quad_size,resolution,time
0,0,"[-122.519531233, 37.7185903212, -122.343749983...",https://api.planet.com/basemaps/v1/mosaics/48b...,327-1256,https://api.planet.com/basemaps/v1/mosaics/48b...,48bd68d0-011d-4d00-92f5-ffce9432cda8,100,https://api.planet.com/basemaps/v1/mosaics/48b...,https://tiles.planet.com/basemaps/v1/planet-ti...,0,{'_self': 'https://api.planet.com/basemaps/v1/...,"[-180, -57, 180, 68]",EPSG:3857,byte,2018-01-01T00:00:00.000Z,"{'quad_size': 4096, 'resolution': 4.7773142678...",48bd68d0-011d-4d00-92f5-ffce9432cda8,1 mon,"['PSScene3Band', 'REOrthoTile']",2018-02-01T00:00:00.000Z,15,global_monthly_2018_01_mosaic,timelapse,True,https://api.planet.com/basemaps/v1/mosaics/48b...,https://api.planet.com/basemaps/v1/mosaics/48b...,https://tiles.planet.com/basemaps/v1/planet-ti...,4096,4.777314,2018-01-01 00:00:00+00:00
2,0,"[-122.519531233, 37.7185903212, -122.343749983...",https://api.planet.com/basemaps/v1/mosaics/b03...,327-1256,https://api.planet.com/basemaps/v1/mosaics/b03...,b03cedd7-a2c8-4c12-a273-a70557592d89,100,https://api.planet.com/basemaps/v1/mosaics/b03...,https://tiles.planet.com/basemaps/v1/planet-ti...,1,{'_self': 'https://api.planet.com/basemaps/v1/...,"[-180, -85.051129, 180, 85.051129]",EPSG:3857,byte,2018-02-01T00:00:00.000Z,{'quad_pattern': 'L{glevel:d}-{tilex:04d}E-{ti...,b03cedd7-a2c8-4c12-a273-a70557592d89,1 mon,[''],2018-03-01T00:00:00.000Z,15,global_monthly_2018_02_mosaic,timelapse,True,https://api.planet.com/basemaps/v1/mosaics/b03...,https://api.planet.com/basemaps/v1/mosaics/b03...,https://tiles.planet.com/basemaps/v1/planet-ti...,4096,4.777314,2018-02-01 00:00:00+00:00
4,0,"[-122.519531233, 37.7185903212, -122.343749983...",https://api.planet.com/basemaps/v1/mosaics/0d8...,327-1256,https://api.planet.com/basemaps/v1/mosaics/0d8...,0d8ee4dc-68de-4dd0-94be-34261e5b46c8,100,https://api.planet.com/basemaps/v1/mosaics/0d8...,https://tiles.planet.com/basemaps/v1/planet-ti...,2,{'_self': 'https://api.planet.com/basemaps/v1/...,"[-180, -85.051129, 180, 85.051129]",EPSG:3857,byte,2018-03-01T00:00:00.000Z,{'quad_pattern': 'L{glevel:d}-{tilex:04d}E-{ti...,0d8ee4dc-68de-4dd0-94be-34261e5b46c8,1 mon,[''],2018-04-01T00:00:00.000Z,15,global_monthly_2018_03_mosaic,timelapse,True,https://api.planet.com/basemaps/v1/mosaics/0d8...,https://api.planet.com/basemaps/v1/mosaics/0d8...,https://tiles.planet.com/basemaps/v1/planet-ti...,4096,4.777314,2018-03-01 00:00:00+00:00
6,0,"[-122.519531233, 37.7185903212, -122.343749983...",https://api.planet.com/basemaps/v1/mosaics/f5d...,327-1256,https://api.planet.com/basemaps/v1/mosaics/f5d...,f5d535a8-3709-4198-a867-e6106c35e9a7,100,https://api.planet.com/basemaps/v1/mosaics/f5d...,https://tiles.planet.com/basemaps/v1/planet-ti...,3,{'_self': 'https://api.planet.com/basemaps/v1/...,"[-180, -56, 180, 76]",EPSG:3857,byte,2018-04-01T00:00:00.000Z,"{'quad_size': 4096, 'resolution': 4.7773142678...",f5d535a8-3709-4198-a867-e6106c35e9a7,1 mon,"['PSScene3Band', 'REOrthoTile']",2018-05-01T00:00:00.000Z,15,global_monthly_2018_04_mosaic,timelapse,True,https://api.planet.com/basemaps/v1/mosaics/f5d...,https://api.planet.com/basemaps/v1/mosaics/f5d...,https://tiles.planet.com/basemaps/v1/planet-ti...,4096,4.777314,2018-04-01 00:00:00+00:00
8,0,"[-122.519531233, 37.7185903212, -122.343749983...",https://api.planet.com/basemaps/v1/mosaics/a2c...,327-1256,https://api.planet.com/basemaps/v1/mosaics/a2c...,a2c10c23-54c1-45e1-8a0a-50b83cfc3d77,100,https://api.planet.com/basemaps/v1/mosaics/a2c...,https://tiles.planet.com/basemaps/v1/planet-ti...,4,{'_self': 'https://api.planet.com/basemaps/v1/...,"[-180, -56, 180, 76]",EPSG:3857,byte,2018-05-01T00:00:00.000Z,"{'quad_size': 4096

## 02. Lazily Load in a single tiff as an Xarray DataArray

In [4]:
import xarray as xr
def create_multiband(row, chunks={'x': 4096, 'y': 4096}):
    url = row['download_link']
    da = xr.open_rasterio(url, chunks=chunks)
    da.attrs['download_url'] = row['download_link']
    da.attrs['mosaic_id'] = row['mosaic_id']
    da.attrs['time'] = row['time']
    da.attrs['quad_id'] = row['id_quad']
    return da

In [6]:
a_row = single_quad_df.iloc[0]
da = create_multiband(a_row)
da

<xarray.DataArray (band: 4, y: 4096, x: 4096)>
dask.array<shape=(4, 4096, 4096), dtype=uint8, chunksize=(4, 4096, 4096)>
Coordinates:
  * band     (band) int64 1 2 3 4
  * y        (y) float64 4.559e+06 4.559e+06 4.559e+06 ... 4.54e+06 4.54e+06
  * x        (x) float64 -1.364e+07 -1.364e+07 ... -1.362e+07 -1.362e+07
Attributes:
    transform:     (4.77731426716, 0.0, -13638811.829080032, 0.0, -4.77731426...
    crs:           EPSG:3857
    res:           (4.77731426716, 4.77731426716)
    is_tiled:      1
    nodatavals:    (nan, nan, nan, nan)
    download_url:  https://api.planet.com/basemaps/v1/mosaics/48bd68d0-011d-4...
    mosaic_id:     48bd68d0-011d-4d00-92f5-ffce9432cda8
    time:          2018-01-01 00:00:00+00:00
    quad_id:       327-1256

## 02.1 Visualize DataArray

In [7]:
# You'll probably need to rerun this cell twice for some silly reason
import hvplot.xarray
import hvplot.pandas  
import cartopy.crs as ccrs
import holoviews as hv
from holoviews.operation.datashader import regrid, shade

In [8]:
%%opts RGB [width=600 height=600]
regrid(da.hvplot(crs= ccrs.Mercator.GOOGLE))

:DynamicMap   []
   :RGB   [x,y]   (R,G,B)

## 03. Lazily Load a stack of quad tiffs into a concatenated DataArray

In [11]:
def stack_dataarrays(dataarrays):
    das = []
    for da in dataarrays:
        da = da.squeeze().drop(labels='band')
        da.attrs['time'] = da.time
        das.append(da)
    DA = xr.concat(das, dim='time')
    DA.coords['time'] = DA.time
    return DA

In [9]:
arrays = [create_multiband(row) for _, row in single_quad_df.iterrows()]


In [12]:
DA = stack_dataarrays(arrays)
DA

<xarray.DataArray (time: 12, band: 4, y: 4096, x: 4096)>
dask.array<shape=(12, 4, 4096, 4096), dtype=uint8, chunksize=(1, 4, 4096, 4096)>
Coordinates:
  * y        (y) float64 4.559e+06 4.559e+06 4.559e+06 ... 4.54e+06 4.54e+06
  * x        (x) float64 -1.364e+07 -1.364e+07 ... -1.362e+07 -1.362e+07
  * time     (time) int64 0 1 2 3 4 5 6 7 8 9 10 11
Dimensions without coordinates: band
Attributes:
    transform:   (4.77731426716, 0.0, -13638811.829080032, 0.0, -4.7773142671...
    crs:         EPSG:3857
    res:         (4.77731426716, 4.77731426716)
    is_tiled:    1
    nodatavals:  (nan, nan, nan, nan)
    quad_id:     327-1256

## 03.1 Inspect DataArray with Datashader

In [ ]:
shade(regrid(DA.hvplot(crs=ccrs.Mercator.GOOGLE)))

## 03.2 Convert DataArray to Dataset

In [13]:
DS = DA.to_dataset(dim='band')
DS

<xarray.Dataset>
Dimensions:  (time: 12, x: 4096, y: 4096)
Coordinates:
  * y        (y) float64 4.559e+06 4.559e+06 4.559e+06 ... 4.54e+06 4.54e+06
  * x        (x) float64 -1.364e+07 -1.364e+07 ... -1.362e+07 -1.362e+07
  * time     (time) int64 0 1 2 3 4 5 6 7 8 9 10 11
Data variables:
    0        (time, y, x) uint8 dask.array<shape=(12, 4096, 4096), chunksize=(1, 4096, 4096)>
    1        (time, y, x) uint8 dask.array<shape=(12, 4096, 4096), chunksize=(1, 4096, 4096)>
    2        (time, y, x) uint8 dask.array<shape=(12, 4096, 4096), chunksize=(1, 4096, 4096)>
    3        (time, y, x) uint8 dask.array<shape=(12, 4096, 4096), chunksize=(1, 4096, 4096)>
Attributes:
    transform:   (4.77731426716, 0.0, -13638811.829080032, 0.0, -4.7773142671...
    crs:         EPSG:3857
    res:         (4.77731426716, 4.77731426716)
    is_tiled:    1
    nodatavals:  (nan, nan, nan, nan)
    quad_id:     327-1256

## 04. Compute NDVI for all timepoints 

In [ ]:
NDVI = (DS[3] - DS[0]) / ( DS[3] + DS[0])

In [ ]:
regrid(NDVI[0].hvplot(crs= ccrs.Mercator.GOOGLE))

## 04.1 Calculate mean and sum of NDVI across all timepoints

In [ ]:
mean_avg = NDVI.mean(dim='time')
summed= NDVI.sum(dim='time')


In [ ]:
mean_plot = regrid(mean_avg.hvplot(crs= ccrs.Mercator.GOOGLE))
summed_plot = regrid(summed.hvplot(crs= ccrs.Mercator.GOOGLE))

mean_plot + summed_plot



## 04.2 Calculate NDVI difference between first and last month in stack

In [ ]:
difference = NDVI[-1] - NDVI[0]

In [ ]:
# regrid(difference.hvplot(crs= ccrs.Mercator.GOOGLE)
difference.plot.imshow()